Classe Board e funções relevantes associadas
Tabuleiro connect four é uma matriz 6x7, onde os espaços vazios são indicados por "-"

In [15]:
from copy import deepcopy

class Board:
    def __init__(self , player=None) -> None:
        self.board = []
        self.player = player

        for _ in range(6):
            row = []
            for _ in range(7):
                row.append('-')
            self.board.append(row)
        
    def __str__(self) -> str:
        string = ''
        for i in range(7):
            string += ''.join(str(i) + " ") 
        string+= '\n'
        
        for row in range(6):  
            string += ' '.join(self.board[row]) + '\n'
        return string
    
    def __eq__(self , other) -> bool:
        for row in range(6):
            if self.board[row] != other.getRow(row):
                return False
        return True    
    
    def getRow(self , row:int) -> list:
        return self.board[row]
    
    def getPos(self , row:int , col:int) -> str:
        return self.board[row][col]
    
    def setPos(self , row:int , col:int) -> None:
        self.board[row][col] = self.player
        self.player = 'X' if self.player == 'O' else 'O'

    def resetBoard(self) -> None:
        for row in range(6):
            for col in range(7):
                self.board[row][col] = '-'

    def boardCopy(self):
        # se faz uma copia do tabuleiro no estado atual
        copy = Board()
        copy.board = deepcopy(self.board)
        copy.player = self.player
        return copy
    
    def finished(self) -> str | bool:
        for line in range(6):
            for col in range(7):
                if (line == 0):
                    row = self.getRow(line)
                    if (row.count('-') == 0): return 'Tie'
                    
                #horizontal
                if col <= 3:
                    if (self.getPos(line, col) == self.getPos(line, col + 1) == self.getPos(line, col + 2) == self.getPos(line, col + 3) and self.getPos(line, col) != '-'):
                        return self.getPos(line, col)
                #vertical
                if line <= 2:
                    if (self.getPos(line, col) == self.getPos(line + 1, col) == self.getPos(line + 2, col) == self.getPos(line + 3, col) and self.getPos(line, col) != '-'):
                        return self.getPos(line, col)
                #diagonal
                if (col <= 3 and line <= 2):
                    if (self.getPos(line, col) == self.getPos(line + 1, col + 1) == self.getPos(line + 2, col + 2) == self.getPos(line + 3, col + 3) and self.getPos(line, col) != '-'):
                        return self.getPos(line, col)
                #diagonal
                if (col <= 3 and line >= 3):
                    if (self.getPos(line, col) == self.getPos(line - 1, col + 1) == self.getPos(line - 2, col + 2) == self.getPos(line - 3, col + 3) and self.getPos(line, col) != '-'):
                        return self.getPos(line, col)
        return False


Na célula abaixo estão as funções relacionadas a funcionalidade do jogo e adquirir informações relevantes para os algoritmos

In [16]:
import random

def askForFirstPlayer() -> list:
    while True:
        piece = input("Queres ser X, O or R? ")
        if piece.upper() == "X":
            return ['X', 'O']
        elif piece.upper() == "O":
            return ['O', 'X']
        elif piece.upper() == "R":
            m = random.randint(0, 1)
            return ['X', 'O'] if m == 0 else ['O', 'X']

def askForAlgorithm() -> int:
    while True:
        try:
            algorithm = int(input("Qual algoritmo queres? 1 - PvsP, 2 - A*, 3 - MonteCarlo, 4 - Minimax: "))
        except ValueError:
            print("Tente inteiros. \n")
            continue

        if algorithm in range(1, 5, 1):
            return algorithm
        else:
            print('Tente um número de 1 a 4. \n')
        
def playAgain() -> bool:
    while True:
        play = input(("Queres jogar de novo? S ou N. : "))
        if play.upper() == 'S':
            return True
        elif play.upper() == 'N':
            return False
        else:
            print('Tente S ou N. \n')

def testMove(board:Board , col:int) -> int:
    for i in range(6):
        if board.getPos(5-i, col) == "-":
            return 5 - i
    return -1

def possibleMoves(board:Board) -> list:
    acc = []
    for col in range(7):
        line = testMove(board, col)
        if line != -1:
            acc.append((line, col))
    return acc

def askForNextMove(board:Board) -> tuple:
    while True:
        try:
            col = int(input("Em qual coluna? "))
        except ValueError:
            print("Tente inteiros.")
            continue
        
        if col > 6 or col < 0:
            print("Fora das margens.")
        else:
            line = testMove(board, col)
            
            if line == -1:
                print("Coluna cheia.")
            else:
                board.setPos(line, col)
                return (line, col)

def winnerAi(board:Board , order:list) -> bool: 
    win = board.finished()
    if isinstance(win, str):
        if win == 'Tie':
            print('Empate.')
        elif win == order[0]:
            print('Ganhaste.')
        else:
            print('A IA ganhou.')
        return True
    return False


Implementação do modo de jogo Player vs Player 

In [17]:
def winnerPvsP(board:Board) -> bool:
    win = board.finished()
    if isinstance(win, str):
        if win == 'Tie':
            print('Empate')
        else:
            print('O vencedor é ' + win + '.')
        return True
    return False

def gamePvsP(board):
    print(board)
    while True:   
        print('Jogador ' + board.player + '.')     
        askForNextMove(board)
        print(board)
        
        if winnerPvsP(board):
            return 

Na célula abaixo temos a implementação do A*, o qual o foi testado duas possibilidades, na função "get_line_points_original", pode-se observar a utilização do sistema de pontos habitual, comumente utilizado para problemas não adversarial. Facilmente se nota que o algoritmo no estado original, não costuma fazer boas escolhas, o que por consequencia temos um algoritmo inadequado para solução de problemas complicados. 

Em get_line_points_modidfied, foram feitas alterações para que o algoritmo fosse mais eficiente em suas escolhas adaptadas ao connect four. Após as modificações por mais que tecnicamente se descaracterizasse ser o algoritmo A*, pois nesse caso os nós não estritamente crescentes no sentido amplo. Pois foi necessario atribuir pesos maiores para impedir jogadas adversárias.

In [18]:

def get_line_points_original(line:list, player:str) -> int:

    if (line.count('X') == 3 and line.count('O') == 0): 
        return 50
    if (line.count('X') == 2 and line.count('O') == 0): 
        return 10
    if (line.count('X') == 1 and line.count('O') == 0): 
        return 1
    if (line.count('X') == 0 and line.count('O') == 3):
        return -50
    if (line.count('X') == 0 and line.count('O') == 2):
        return -10
    if (line.count('X') == 0 and line.count('O') == 1): 
        return -1
    return 0

def get_line_points_modidfied(line:list , player:str) -> int:
    opponent = 'X' if player == 'O' else 'O'
    if (line.count('X') == 3 and line.count('O') == 0): 
        if 'X' == opponent: return 250
        else: return 50
    if (line.count('X') == 2 and line.count('O') == 0): 
        if 'X' == opponent: return 50
        else: return 10
    if (line.count('X') == 1 and line.count('O') == 0): 
        return 1
    if (line.count('X') == 0 and line.count('O') == 3):
        if 'O' == opponent: return -250
        else: return -50
    if (line.count('X') == 0 and line.count('O') == 2):
        if 'O' == opponent: return -50
        else: return -10
    if (line.count('X') == 0 and line.count('O') == 1): 
        return -1
    return 0

def getPoints(board:Board , player:str , getLinePoints) -> int:
    win = board.finished()
    if win == 'X': return 512
    if win == 'O': return -512
    if win == 'Tie': return 0
    points = 16 if player == 'X' else -16
    for line in range(6):
        for col in range(7):
            #horizontal
            if col < 4:
                points += getLinePoints( [board.getPos(line, col+i) for i in range(4)], player)

            #vertical
            if line < 3:
                points += getLinePoints( [board.getPos(line + i, col) for i in range(4)], player)
                
            #diagonal positiva
            if line < 3 and col < 4:
                points += getLinePoints( [board.getPos(line + i, col + i) for i in range(4)], player)

            #diagonal negativa
            if line > 2 and col < 4:
                points += getLinePoints( [board.getPos(line - i, col + i) for i in range(4)], player)
    return points

def Astar(state:Board , getLinePoints) -> list:
    ai = state.player
    moves = possibleMoves(state)
    best_move = [state, getPoints(state, ai, getLinePoints), -1, -1]
    for line, col in moves:
        new_state = state.boardCopy()
        new_state.setPos(line, col)
        new_state_points = getPoints(new_state, ai, getLinePoints)
        if ai == 'X':
            if new_state_points == 512:
                return [new_state, line, col]
            if new_state_points > best_move[1]:
                best_move = [new_state, new_state_points, line, col]
        else:
            if new_state_points == -512:
                return [new_state, line, col]
            if new_state_points < best_move[1]:
                best_move = [new_state, new_state_points, line, col]
    
    #para verificar se ela não joga         
    if state == best_move[0]:
        print('-------------------------------------------------------------------------------------------')
        
    return [best_move[0], best_move[2], best_move[3]]

def gameAstar(board:Board , order:list) -> None:
    while True: 
        try:
            option = int(input('Queres jogar com que heurística? 1 - Original, 2 - Modificada: '))
        except ValueError:
            print('Tente 1 ou 2. \n')
            continue
        if option == 1:
            get_line_points = get_line_points_original
            break
        elif option == 2:
            get_line_points = get_line_points_modidfied
            break
        else:
            print('Tente 1 ou 2. \n')
    print(board)
    while True:
        print('Tua vez.')
        _, col = askForNextMove(board)
        print(board)
        
        if winnerAi(board, order):
            return
        
        board, _, col = Astar(board, get_line_points)
        print('A IA pôs uma peça na coluna ' + str(col) + '.')
        print(board)
        
        if winnerAi(board, order):
            return

Abaixo temos a implementação do algoritmo minimax com alpha beta cut, esse algoritmo se mostrou altamente adequado para a criação de um adversário inteligente no jogo connect four. 

Após um entendimento do significado do que é jogar de maneira inteligente no connect four, foi notado algumas caracteristicas importantes:
* O jogador com mais peças na posição central, possui mais possibilidades de fazer 4 em linha.
* Para se sagrar vitorioso é de fundaental importância impedir que o adversário ponha 3 peças em 4 espaços vazios.
* Para se ter mais possibilidades de vitória é bom impedir que o adversário ocupe 2 peças em 4 espaços vazios, entretanto é preferível termos 2 peças em 4 espaços vazios.

A uma profundidade superior a 5 (inclusive), o algortimo se mostra bastante eficiente e com jogadas capaz de vencer qualquer estudante da FCUP, mas admiro aos que tiverem coragem de tentar.

In [19]:
def scoreLine(line:list , player:str , number_pieces:int) -> int:
    if line.count(player) == number_pieces and line.count('-') == 4 - number_pieces:
        return 1
    return 0

def centrais(board:Board , player:str , n:int) -> int:
    points = 0
    for line in range(6):
        if board.getPos(line, 4) == player and n == 4:
            points += 1
        if board.getPos(line, 3) == player or board.getPos(line, 5) == player and n == 3:
            points += 1
    return points

def score_position(board:Board , player:str , number_pieces:int) -> int:
    points = 0
    for line in range(6):
        for col in range(7):
            #horizontal
            if col < 4:
                points += scoreLine( [board.getPos(line, col+i) for i in range(4)], player, number_pieces)
            #vertical
            if line < 3:
                points += scoreLine( [board.getPos(line + i, col) for i in range(4)], player, number_pieces)     
            #diagonal positiva
            if line < 3 and col < 4:
                points += scoreLine( [board.getPos(line + i, col + i) for i in range(4)], player, number_pieces)
            #diagonal negativa
            if line > 2 and col < 4:
                points += scoreLine( [board.getPos(line - i, col + i) for i in range(4)], player, number_pieces)
    return points

def heuristica(board:Board , order:list) -> int:
    central = 10 * (centrais(board, order[0], 4) - centrais(board, order[1], 4))
    neigh_center = 8 * centrais(board, order[0], 3) - centrais(board, order[1], 3)
    strong_line = 100 * (score_position(board, order[0], 4) - score_position(board, order[1], 4))
    medium_line = 20 * score_position(board, order[0], 3) - 40 * score_position(board, order[1], 3)
    weak_line =  5 * score_position(board, order[0], 2) - score_position(board, order[1], 2)
    return  central + neigh_center + strong_line + medium_line + weak_line
    
def bestMove(board:Board , depth:int , alpha, beta, maximizing:bool , order:list):
    possible_moves = possibleMoves(board)
    terminal_node = isinstance(board.finished(), str)

    if depth == 0 or terminal_node:
        return None, heuristica(board, order)

    if maximizing:
        max_value = float('-inf')
        best_move = None

        for line, col in possible_moves:
            copy = board.boardCopy()
            copy.setPos(line, col)
            _, new_score = bestMove(copy, depth - 1, alpha, beta, False, order)

            if new_score > max_value:
                max_value = new_score
                best_move = (line, col)
            alpha = max(alpha, max_value)

            if alpha >= beta:
                break
        return best_move, max_value
        
    else:
        min_value = float('inf')
        best_move = None

        for line, col in possible_moves:
            copy = board.boardCopy()
            copy.setPos(line, col)
            _, new_score = bestMove(copy, depth - 1, alpha, beta, True, order)
            if new_score < min_value:
                min_value = new_score
                best_move = (line, col)
            beta = min(beta, min_value)
            if alpha >= beta:
                break
    return best_move, min_value
    
def gameMiniMax(board:Board , order:list) -> None:
    while True: 
        try:
            depth = int(input('Qual a profundidade queres? '))
        except ValueError:
            print('Tente inteiros entre 1 e 10. \n')
            continue
        if depth in range(1,11,1):
            break
        else:
            print('Tente inteiros entre 1 e 10. \n')
    print(board)
    while True:
        print('Tua vez.')
        askForNextMove(board)
        print(board)
        
        if winnerAi(board, order):
            return 
        
        (line, col), *_ = bestMove(board, depth, float('-inf'), float('inf'), True, order)
        board.setPos(line, col)
        print('A IA pôs uma peça na coluna ' + str(col) + '.')
        print(board)
        
        if winnerAi(board, order):
            return

In [20]:
from numpy import sqrt, log as ln
import time

class Node:
    def __init__(self , state) -> None:
        self.state = state
        self.parent = None
        self.children = []
        self.c = sqrt(2)
        self.visits = 0
        self.wins = 0
    
    def __str__(self) -> str:
        string = "Estado: " + str(type(self.state)) + '\n'
        string += "Pai: " + str(self.parent != None) + '\n'
        string += "Filhos: " + str(len(self.children)) + '\n'
        string += "Vitórias: " + str(self.wins) + '\n'
        string += "Total: " + str(self.visits) + '\n'
        string += "Pontuação: " + str(self.uct()) + '\n'
        string += "Probabilidade de vitória: " + str(self.win_probability()) + '\n'
        return string
    
    def add_child(self, child) -> bool:
        #se o estado já estiver como filho
        if any(child.state == node.state for node in self.children):
            return False

        self.children.append(child)
        child.parent = self
        return True
    
    def add_children(self , children:list) -> None:
        for child in children:
            self.add_child(child)

    def uct(self) -> float:
        if self.visits == 0:
            return float('inf')
        exploitation = self.wins / self.visits
        exploration = self.c * sqrt(2 * ln(self.parent.visits) / self.visits) if self.parent else 0
        return exploitation + exploration
    
    def win_probability(self) -> float:
        return self.wins / self.visits
    
class MCTS:
    def __init__(self , root:Node) -> None:
        self.root = root

    def best_child(self , node:Node) -> Node:
        best_child = []
        best_score = float('-inf')
        for child in node.children:
            score = child.uct()
            if score > best_score:
                best_child = [child]
                best_score = score
            elif score == best_score:
                best_child.append(child)
        return random.choice(best_child)
    
    def biggest_win_probability(self):
        best_child = []
        best_score = float('-inf')
        for child in self.root.children:
            score = child.win_probability()
            if score > best_score:
                best_child = [child]
                best_score = score
            elif score == best_score:
                best_child.append(child)
        return random.choice(best_child)

    def update_state(self , state:Board) -> None:
        self.root = Node(state)
        
    def select(self) -> Node:
        node = self.root
        while len(node.children) > 0:
            node = self.best_child(node)
        return node

    def expand(self , node:Node) -> Node:
        child_moves = possibleMoves(node.state)
        
        for line, col in child_moves:
            child_state = node.state.boardCopy()
            child_state.setPos(line, col)
            node.add_child(Node(child_state))
        return random.choice(node.children)
        
    def rollout(self , node:Node) -> str:
        state = node.state.boardCopy()
        while state.finished() == False:
            state = self.rollout_policy(state)
        return state.finished()

    def rollout_policy(self , state:Board):
        line, col = random.choice(possibleMoves(state))
        state.setPos(line, col)
        return state

    def back_propagation(self , node:Node , winner_symbol:str) -> None:
        while node:
            node.visits += 1
            if winner_symbol != node.state.player and winner_symbol != 'Tie':
                node.wins += 1
            node = node.parent

    def search(self , max_time:int) -> Node:
        start_time = time.time()
        simulations = 0
        while time.time() - start_time < max_time:
            simulations += 1
            selected = self.select()
            result = selected.state.finished() 
            if isinstance(result, bool): 
                expanded = self.expand(selected)
                result = self.rollout(expanded)
            self.back_propagation(selected, result)
            
        print('Foram feitas ' + str(simulations) + ' simulações.')
        return self.biggest_win_probability()

def gameMonteCarlo(board:Board , order:list) -> None:
    print(board)
    m = MCTS(board)
    while True:
        print('Tua vez.')
        askForNextMove(board)
        m.update_state(board)
        print(board)
        
        if winnerAi(board, order):
            return
        
        board = m.search(10).state
        print(board)
        
        if winnerAi(board, order):
            return


In [21]:
def main():
    play = True
    while play:
        order = askForFirstPlayer()
        board = Board(order[0])
        board.resetBoard()
        game = askForAlgorithm()
        if game == 1:
            print("Escolhido Player vs Player.", end="\n")
            gamePvsP(board)
        if game == 2:
            print("Escolhido A*.", end="\n")
            gameAstar(board, order)
        if game == 3:
            print("Escolhido MonteCarlo", end="\n")
            gameMonteCarlo(board, order)
        if game == 4:
            print("Escolhido Minimax", end='\n')
            gameMiniMax(board, order)

        play = playAgain()


if __name__ == '__main__':
    main()

Escolhido Player vs Player.
0 1 2 3 4 5 6 
- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -

Jogador X.
0 1 2 3 4 5 6 
- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -
- X - - - - -

Jogador O.
0 1 2 3 4 5 6 
- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -
- X O - - - -

Jogador X.
0 1 2 3 4 5 6 
- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -
- X - - - - -
- X O - - - -

Jogador O.
0 1 2 3 4 5 6 
- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -
- X O - - - -
- X O - - - -

Jogador X.
0 1 2 3 4 5 6 
- - - - - - -
- - - - - - -
- - - - - - -
- X - - - - -
- X O - - - -
- X O - - - -

Jogador O.
0 1 2 3 4 5 6 
- - - - - - -
- - - - - - -
- - - - - - -
- X O - - - -
- X O - - - -
- X O - - - -

Jogador X.
0 1 2 3 4 5 6 
- - - - - - -
- - - - - - -
- X - - - - -
- X O - - - -
- X O - - - -
- X O - - - -

O vencedor é X.
